In [3]:
#### Testing Regressions
#pip install linearmodels
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("analysis_state_years.csv", header=0)

hurricane_dict = {"Harvey": (["22","48"],2017),
                        "Irma": (["01","12","13","28","45","47"],2017),
                        "Sandy": (["09","10","11","24","34","36","42","51","54"],2012),
                        "Maria": (["72"],2017)}

variable_dict = {'population':'County-level population.',
                'foreign_born':'Percentage of county population born outside U.S.',
                'black_afam':'Percentage of county population Black/African American.',
                'median_income':'Median household income (in nominal dollars).',
                'snap_benefits':'Percentage of county households on SNAP benefits in the last 12 months.',
                'unemp_rate':'County unemployment rate for prime-age workers.',
                'health_insurance_rate':'Percentage of county population with health insurance coverage.',
                'vacant_housing_rate':'Percentage of housing units vacant at time of survey.',
                'rental_vacancy_rate':'Percentage of rental units vacant at time of survey.',
                'median_rent':'Median gross rent at county level (in nominal dollars).',
                'median_home_price':'Median home price at county level (in nominal dollars).'}

states,year = hurricane_dict["Irma"]

states=[9,10,11,24,34,36,42,51,54]
data = data.loc[data['state_fips'].isin(states)]
data = data.loc[data['year'] == year]

def vif_detection(exog_vars,dep_var):
        '''
        Method computing Variance Inflation Factors (VIF) on prospective exogenous variables for regression. 
        5 (inclusive) is used as the cutoff for multicollinearity. This function eliminiates the variable with 
        the highest VIF, and reruns the regression until the highest VIF is below the threshold.

        Input:
            -exog_vars (pandas df): pandas dataframe of potential exogenous variables for regression.
            -dep_var (pandas series): pandas dataframe of dependent variable in regression.
        '''
        max_vif = float('inf')
        while max_vif > 5:
            reg_string = dep_var.columns[0] + ' ~ ' + '+'.join(exog_vars.columns)
            _, X = dmatrices(reg_string, data=data, return_type='dataframe')
            vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
            max_vif = max(vif[1:])
            if max_vif > 5:
                max_col = vif.index(max_vif)
                exog_vars = exog_vars.drop(exog_vars.columns[max_col-1], axis=1)

        return exog_vars

In [5]:
# Panel Reg-Output at the end
y = pd.DataFrame(data, columns=['aid_requested'])
exog = pd.DataFrame(data, columns=['foreign_born','black_afam','median_income','snap_benefits','unemp_rate','health_insurance_rate','vacant_housing_rate','rental_vacancy_rate','median_rent','median_home_price','population'])
exog_vars = vif_detection(exog,y)
X = sm.add_constant(exog_vars)
pooled_reg = sm.OLS(y,X).fit()

coefs = pooled_reg.params
std_err = pooled_reg.bse
tvals = pooled_reg.tvalues
pvals = pooled_reg.pvalues

results_df = pd.DataFrame({"Coefficient Estimate":coefs, "Standard Error":std_err, "T-Stat":tvals, "P-Value":pvals})

results_df.round(decimals=3)

,Coefficient Estimate,Standard Error,T-Stat,P-Value
const,-4.050888e+08,2.915255e+08,-1.390,0.173
foreign_born,-9.123442e+05,1.930964e+06,-0.472,0.639
black_afam,3.405902e+05,6.456343e+05,0.528,0.601
median_income,-1.024191e+03,1.746078e+03,-0.587,0.561
snap_benefits,1.766262e+06,2.159083e+06,0.818,0.419
unemp_rate,-7.766948e+04,7.809777e+06,-0.010,0.992
health_insurance_rate,4.527181e+06,3.339583e+06,1.356,0.183
vacant_housing_rate,-5.502432e+05,1.054856e+06,-0.522,0.605
rental_vacancy_rate,-8.326668e+04,1.593256e+06,-0.052,0.959
population,1.619700e+01,3.088400e+01,0.524,0.603


In [6]:
#Create variable table
desc_list=[]
for var in exog_vars.columns:
    desc_list.append(variable_dict[var])

var_df = pd.DataFrame({"Independent Variable":exog_vars.columns, "Description":desc_list})
var_df

,Independent Variable,Description
0,foreign_born,Percentage of county population born outside U.S.
1,black_afam,Percentage of county population Black/African ...
2,median_income,Median household income (in nominal dollars).
3,snap_benefits,Percentage of county households on SNAP benefi...
4,unemp_rate,County unemployment rate for prime-age workers.
5,health_insurance_rate,Percentage of county population with health in...
6,vacant_housing_rate,Percentage of housing units vacant at time of ...
7,rental_vacancy_rate,Percentage of rental units vacant at time of s...
8,population,County-level population.


In [7]:
# Data from regression
reg_data = y.merge(exog_vars, left_index=True, right_index=True)
reg_data

,aid_requested,foreign_born,black_afam,median_income,snap_benefits,unemp_rate,health_insurance_rate,vacant_housing_rate,rental_vacancy_rate,population
0,0.000000e+00,6.422354,2.0,41280.0,7.3,2.0,97.7,8.7,4.4,229869
1,0.000000e+00,9.680841,11.2,29630.0,13.4,6.2,96.4,29.7,1.4,155565
2,0.000000e+00,9.680841,11.2,29630.0,13.4,6.2,96.4,29.7,1.4,155565
3,0.000000e+00,15.758701,8.3,38558.0,6.4,2.5,95.5,16.3,6.2,1492953
4,0.000000e+00,11.252567,10.0,27231.0,15.7,5.9,96.2,48.9,14.2,75485
5,0.000000e+00,13.895727,3.6,25000.0,10.1,1.9,95.9,7.4,1.1,104802
6,0.000000e+00,8.199335,5.7,30715.0,12.2,3.2,93.8,19.2,3.8,179417
7,5.216897e+05,8.529448,4.1,25323.0,7.5,2.3,96.0,12.1,7.0,162660
8,0.000000e+00,2.048471,5.0,25801.0,15.5,2.8,94.0,13.7,12.3,113841
9,0.000000e+00,2.048471,5.0,25801.0,15.5,2.8,94.0,13.7,12.3,113841
